In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import h5py
import numpy as np
import json
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Image, display, clear_output
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
sns.set_style('whitegrid')

In [6]:
from keras import optimizers
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense, Input
from keras.regularizers import l2, l1
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, History
from keras import backend as K
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import load_img

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array

In [8]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    model = VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(train_data_dir, target_size=(img_width,img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_train = model.predict(generator, nb_train_samples//batch_size)
    np.save(location+'/bottleneck_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir, target_size=(img_width,img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_validation = model.predict(generator, nb_validation_samples//batch_size)
    np.save(location+'/bottleneck_features_validation.npy', bottleneck_features_validation)

In [9]:
def train_categorical_model():
    train_data = np.load(location+'/bottleneck_features_train.npy')
    train_labels = np.array([0]*(278) + [1]*(315) + [2]*(386))
    train_labels = to_categorical(train_labels)

    validation_data = np.load(location+'/bottleneck_features_validation.npy')
    validation_labels = np.array([0]*(48) + [1]*(55) + [2]*(68))
    validation_labels = to_categorical(validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(top_model_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True,mode='auto')

    fit = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(validation_data,validation_labels), callbacks=[checkpoint])

    with open(location+'/top_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model,fit.history

In [10]:
def finetune_categorical_model():
    input_tensor = Input(shape=(256,256,3))
    base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
    print("Model loaded.")
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(3, activation='softmax'))

    top_model.load_weights(top_model_weights_path)

    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

    for layer in model.layers[:25]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.00001, momentum=0.9), metrics=['accuracy'])

    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    checkpoint = ModelCheckpoint(fine_tuned_model_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

    fit = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size, verbose=1, callbacks=[checkpoint])

    with open(location+'/ft_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model, fit.history

In [11]:
def print_best_model_results(model_hist):
    best_epoch = np.argmax(model_hist['val_accuracy'])
    print('epoch:', best_epoch+1, \
    ', val_accuracy:', model_hist['val_accuracy'][best_epoch], \
    ', val_loss:', model_hist['val_loss'][best_epoch])

In [12]:
def plot_metrics(hist, stop=50):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

    axes = axes.flatten()

    axes[0].plot(range(stop), hist['accuracy'], label='Training', color='#FF533D')
    axes[0].plot(range(stop), hist['val_accuracy'], label='Validation', color='#03507E')
    axes[0].set_title('Accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(loc='lower right')

    axes[1].plot(range(stop), hist['loss'], label='Training', color='#FF533D')
    axes[1].plot(range(stop), hist['val_loss'], label='Validation', color='#03507E')
    axes[1].set_title('Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(loc='upper right')

    plt.tight_layout();

    print("Best Model:")
    print_best_model_results(hist)

In [13]:
def evaluate_categorical_model(model, directory, labels):
    datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_directory(directory, target_size=(img_height,img_width), batch_size=batch_size, class_mode='categorical', shuffle=False)

    predictions = model.predict_generator(generator, len(labels))

    pred_labels = [0 if i<0.5 else 1 for i in predictions]

    print('')
    print(classification_report(validation_labels, pred_labels))
    print('')
    cm = confusion_matrix(validation_labels, pred_labels)
    return cm

In [17]:
location = '/content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data3a'
top_model_weights_path = location+'/top_model_weights.h5'
fine_tuned_model_path = location+'/ft_model.h5'

train_data_dir = location+'/training'
validation_data_dir = location+'/validation'
train_samples = [len(os.listdir(train_data_dir+'/'+i)) for i in sorted(os.listdir(train_data_dir))]
nb_train_samples = 979
validation_samples = [len(os.listdir(validation_data_dir+'/'+i)) for i in sorted(os.listdir(validation_data_dir))]
nb_validation_samples = 171

img_width, img_height = 256,256
#epochs = 50
epochs = 10
batch_size = 16

In [15]:
save_bottleneck_features()

58889256/58889256 [==============================] - 0s 0us/step
Found 979 images belonging to 3 classes.
62/62 [==============================] - 715s 12s/step
Found 171 images belonging to 3 classes.
11/11 [==============================] - 122s 11s/step


In [18]:
d4_model, d4_history = train_categorical_model()

Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 3.9019 - accuracy: 0.4811
Epoch 1: val_accuracy improved from -inf to 0.50877, saving model to /content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data3a/top_model_weights.h5
62/62 [==============================] - 12s 179ms/step - loss: 3.9019 - accuracy: 0.4811 - val_loss: 0.9586 - val_accuracy: 0.5088
Epoch 2/10
62/62 [==============================] - ETA: 0s - loss: 1.2933 - accuracy: 0.5424
Epoch 2: val_accuracy did not improve from 0.50877
62/62 [==============================] - 8s 128ms/step - loss: 1.2933 - accuracy: 0.5424 - val_loss: 3.3519 - val_accuracy: 0.4152
Epoch 3/10
62/62 [==============================] - ETA: 0s - loss: 1.0137 - accuracy: 0.5955
Epoch 3: val_accuracy improved from 0.50877 to 0.56725, saving model to /content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data3a/top_model_weights.h5
62/62 [==============================] - 10s 170ms/step - loss: 1.0137 - accuracy: 0.

In [21]:
#plot_metrics(d4_history)

In [22]:
ft_model, ft_history = finetune_categorical_model()

Model loaded.


Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


<ipython-input-10-aa2ce9dffabc>:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size, verbose=1, callbacks=[checkpoint])


Epoch 1/10
61/61 [==============================] - ETA: 0s - loss: 1.2075 - accuracy: 0.6147 
Epoch 1: val_accuracy improved from -inf to 0.66250, saving model to /content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data3a/ft_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - 837s 14s/step - loss: 1.2075 - accuracy: 0.6147 - val_loss: 1.0335 - val_accuracy: 0.6625
Epoch 2/10
61/61 [==============================] - ETA: 0s - loss: 1.2177 - accuracy: 0.6417 
Epoch 2: val_accuracy did not improve from 0.66250
61/61 [==============================] - 842s 14s/step - loss: 1.2177 - accuracy: 0.6417 - val_loss: 1.0837 - val_accuracy: 0.6438
Epoch 3/10
61/61 [==============================] - ETA: 0s - loss: 1.1541 - accuracy: 0.6303 
Epoch 3: val_accuracy did not improve from 0.66250
61/61 [==============================] - 843s 14s/step - loss: 1.1541 - accuracy: 0.6303 - val_loss: 1.0070 - val_accuracy: 0.6562
Epoch 4/10
61/61 [==============================] - ETA: 0s - loss: 1.1248 - accuracy: 0.6469 
Epoch 4: val_accuracy did not improve from 0.66250
61/61 [==============================] - 844s 14s/step - loss: 1.1248 - accuracy: 0.6469 - val_loss: 1.0820 - val_accuracy: 0.6500
Epoch 5/10
61/61 [====================

In [28]:
ft_model = load_model(location+'/ft_model.h5')

In [29]:
def pipe4(image_path, model):
    urllib.request.urlretrieve(image_path, 'save.jpeg')
    img = load_img('save.jpeg', target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255
    pred = model.predict(x)
    pred_labels = np.argmax(pred, axis=1)
    d = {0:'minor', 1:'moderate', 2:'severe'}
    for key in d.keys():
        if pred_labels[0] == key:
            print("Validating severity of damage....Result:",d[key])
    print("Severity assessment complete.")

In [30]:
pipe4('https://media.istockphoto.com/id/465663510/photo/body-of-car-get-damage-by-accident.jpg?s=612x612&w=0&k=20&c=ltAHdcFKBYJgurn9AqirfPUH0FPAstbz5xIuxFTxUto=', ft_model)

1/1 [==============================] - 1s 813ms/step
Validating severity of damage....Result: moderate
Severity assessment complete.


In [31]:
pipe4('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBUVFBgVFRUZGRgaHB0aHBsbGxwdIR8dIxoaJB0dGyEcIS0kGx0qIRsdJTcmKi4xNTQ0GyM6PzozPi0zNDMBCwsLEA8QHRISHzUqIyozMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzM//AABEIALIBGwMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAAAgMEBQYBBwj/xABBEAACAQIEBAMGBAMGBAcAAAABAhEAAwQSITEFQVFhEyJxBjKBkaGxFMHR8EJSYgcVcoLh8SNDU5IWJDOio7LC/8QAGAEAAwEBAAAAAAAAAAAAAAAAAAECAwT/xAAjEQEBAAICAgMAAgMAAAAAAAAAAQIREiExQQMTUWGhBEJx/9oADAMBAAIRAxEAPwD2aiiigCiim2M6UAsGu0ldqVQBRRRQBRSMw6il0AUUUUAUUjN0roPWgFUUUUAUUUUAUUUUAUUUUAUUU2xnSgFg12kJtS6AKKKKAKKKKAKKKKAKKQW6V0HrQCqKKKAKKKbZ6AGauKvyoVZ9KcoArgM1Re1/FWw+HzIQLjsttCROXMfM3TyoGbXSQBzqr4JxrNg86N5nZlUnXLouZu8EyB3FPXWwuOJ8et2iyjzFfeMwqk8iYMt2AJqVYv8Aj4fOhgupKno3L5EV4n7V+0sP4Ng+5OZzrBPvH+pydz8B29U9gbs4RFLFmCIxJJJll1mecqaAk8KxYuW1aCCRqOmsEfMGrFGrF483LOLZbepS54mTMQHtXAfEGgOqN5xofdI51p3xSlQykwdo6aax01q7Yy1Z2ntcA3ptMSraK0+hqpxuIOWJnkY59fvWXxfiI5dGOWJJB2M8+fKlekfZd6eio3c0eIdpn4fpWX4RxvNo7ajfXXfT1+FXK4nkpkc/9aUsrTn1tYpdPSl+IKhrd7Vz8QswTrT4iZp4cdaVUAvI0NRhj2UwQf32pXFXOLiioQuk+6/wIBH5H60r8Q43UH0MfQ/rS40+UTKKgLxO3MMch6MI+u0fGpIuAjTUdQft1pGUzVxRNAWfSnKAK7RRQBRRRQBRRRQBSG6UukN1oDoFcJ60HtQBQHVpVFFAFN5flTlFAcpFxwASTAAkntTlYP8AtQxQNlMOLjJ4hZ3KmItW1LOT1B0EdSKAie0nHExiPbsvlyPathx1dwxE7A5bTAgSQDrvFY9uJmxwh/DMO19rds8wB70fAR8qd9jrZOCW4dM+LZgOQW3YYBR0ALtWfxj+Jbw1qf8AmXLhHWSp+cIfnTgs9M45KyDvz9a9s/s/x2S8bTHRgwXtGoH0Y/HvXhd+4WLN1k17NguImyhYKo86OGMAnJlGp75efWj0PbQe3CNbvYe8mhbNZJ7kZkB7Ehl+NU9r2iyYkrcUi3et27iESQrjyOh00nJM7Ar3rV+2GFGIwLlNSqi8h7rDD5ifnXlHHMWy2MPikaPDd7bCNGRwLiqQe7ETy151Oux601972hDufCR1STuoGczvrsNt66eLhQoChZ5Rr0O2hFZzA8ft3soEIx3U8usdedTMW9sjMxZejRIBnTPzAiNa1up5rKfHvuJ+NCmLqt4REakZt/4REEz0Eb0nFe0dmwilrjZmBIVUaWg9/dHKT3ql4gbt5lW3GVNQVJMsV0aY2gmI6mqpeCX7l9fE3IgCcxIG8DWN+dZajThGxXjwKB8yhYzZgTBkaRGsjmIkfd6zxDECWLiNgMgI9TmAM+sVmMNw7DYe54iglx1Okie8EjYaR0qxs8UznXMCOWm37+1XjnNp+mRqsLx+NLimeqyB9SasE4zb3lh0Gn5VkFuT+lD3Dzmt8Zjl4Z3G4tNjvaDIJS3ccn+QI2noWH5V2z7SoRLhl5Q9t0+4j4g1mUxHepNrFt1qr8cTutKnF7TwFdPiwNJuYW6ZNi6UPIAKUJ7gCs+9y22j20b1UE/OKQMDhzrlKn+i5cX/AOrCovxqmX60HEeP38Mwz2g9sx5gcrA/1T5fTXntVjwz2nw97y5ij/yuMp+vKsocIIy28RiE9Lmcf/IrVBfgFw+7iQRvD20InqMsAHuAKi4b8Ll17epqwIkGaVXliDH2gBks3gNouXEJg6aMSoPcRWowPFLjW1F+15tCcjAEHeAVIzDl/DNTcKfP9ayioeFxqXPdMH+UiD8jv6iplQsUUUUAUUUUAnLSqKKAKKKKAKKKKA4TXh/9oXEjcfEPOrsmHtjpbQ57nzfID2r2DjeJ8OxcfchTA69qyXs/7DoSt/FjO48y2z7qkwSzDm0jnTDO+zmCf+78KURiIxjEqpPJhbJjaY061hLYhnLGMlm5odCCcq7byJavpYQogAADYCsx7XezuHxtsrcUBwDkuADMh9f4l6qdDS0Le9vGPaWxg1RHtqzuUCnI2RMwAknyyzmT6wPjsuI4QvgbnhqWOV3WPNoVkRG/livOOOZ7SnC3ECtbuMTrOuUAEdiIPxFeo/2WcV8TDLbYyySh9FjL/wC1kH+U1W/xOmj/ALPuMrf4dbz+ZlBtkaGY5+kGPhWSvcKL4bG4M+8jF7e+mUgj45WBHY1aYHAW+HXMQXvLbtOWa2JEqGCGEQSSA2fWANe9H95K9xMRORHYW1BbMboVYJcLojFW2JOw6VlbdtJHkDOmUZiA250OnSIrcPeOFsYU4m4Wa4huZCBITTKDO5giZk/LWs4/wFUx6Wh7l24gAH8rOJ21ghpnvTf9onEFucRbSVtZLYT+EgCSB09407TnSdhuJuLjMVIU+aVIEgbaTz127Gp+EtsHN3xRcJSGVfKon3lk9QNSORiRrNMOLo6+ILZCwQWOgEdeRqTguKWlthdN+2rH3WAOrKDH00qcbub/AKVYm3sO5A5BjJWRJ0MLEAAQSfhUC/ijZJFxSrA+UhdD8R5Z7Cptq/YBYi4CxG7b5p1Hx0NReP27ZABuMzsAyyTAEmIEc43NVot6WVjHJdQMtwI3MEj7GmnxQQz4jOOY5RP3rFLdYHfWrXCY64AZRWA3Lg6dxl1+9V3PCfPlrFM6gyCJB7UoXag8JdfcGx1+Pb1/Kpty3XVh8nKdsM8NU+lzvT2fvVeTAgfP9O1CNWkrOxaJep5MTVQt2nFu0+iXAxXenRi+9UT4iFMb/v50YS/O+vxo1A0aYw8jFWWG4266E5h33+dZnx1O1LS73rPLCXyuZVsf/EVlSouEpnYICQSuY7AsPdnvAq7rzLEv4ltrbAZWEVoPY7jniL4FxszpoG5sByb+ofb41z54cf8AjXHLca6iiioUKKKKAKKKKAKKK4aAznteLngG7baBaYXHWJLIklwP6omPj2IlJinIBBeCOk/HUVYOoZSCAQQZB2IO4PwqgHB8QhyW7qNbGi+IHLAclJB88bToesnUzdqmvaemJuSc4ERodjPSPn0qm43x23ZUtccIO+5/wqNSat7fCXYDPd05hFyz/mYsflFR+I+zli5lTw1iZJIzNMbljJJjmTR2XTyzCYE8XxZKWYT+O64EhRoNt2MaAH6CvTeA8JweFXw0sqjCVLEBiwDEAlxzIIaOWeKmYzwcBhXa2gRVGywCToBrzOtVp46jAMQFt6FXZoYjmQpG2u5P5U7RELj/ALG4e6QzF0JBSVO53UtmmSBPz7CsTxh0tBwC3htKZRHp5ZBAlZE/GtDxbj9rEpbS3ca2gc3FuFIJGQgKF5g5pkwdIiaxr8RtNmL5nCsTGbeNAdtoO0azWWc9NMLrsrD8SF3I6Zi1hcttlAk5WDJmJBHl1AiCRGtVPHuEXblx76nOXYsy+UMDA0AUmREab7aHetdhOP4W0huKQztlAtJlbUnSNwFEjXbTYHSrPhePU5zeVbeniIpQEsDyByjM08hBk7U5ldCx4/ed1t5ZhZnL3jf99qsOB4lEYi9lKx5WnbrBH73qw4/wnKj3LsrcLLkTkVctqD2ysPltNZ1QUJVoGg00PTQj0q8dWbifF7el279tLfiaAgQrDURuInck1U/i0xF46eZRqWYKInrrGv3rK8OxbpCk+SQuYickn3l6VreH+z6qc2cld3J6HlpuT+dPLGednjlfxMfhWHDBWVAxE7kjU/zbj5UYjg3hy9vRQJZTr8R89v8AWr3GojFAmuxAI2EgaHYACdPXpT9hQwYDcqQB1MGljdwZTVZXAtChlGqiY51eW3zqDEHpUIJldtPjy7+nOkJjYaOXM08borNpxSmyK6mKUmJpTxvW2Oe2WWBthXJrjMelRsRjUX3nRf8AE6itpkz4pbbH0pGGMfGqS/7SWF08QHl5QT9Yiog9p0GwYgHLOnTkAdaOeM80cL+NiNNeVJOKUakiKyN3i4uZk/EeEcsiYUfPXWqLxCXT/isSdCWbMoPIyxIg6ctAedZ358fUVPivl6I3GEg+YQNz+uulViYs27gdbko7QMpgo/QHlMHpDf4iRH4rfFu3+HVRlcBHuMGUq2YQcoIDTHTprUTjtothiq6kOp2AmT0HpFZ35OeLSYca9Mw/9oNu0ES/bukkAZ1VSraAFtWBBnUiK22Ax1u8guW3DqdiPsRuD2NeEcF4tZxNkWsQwS4nu3DzmAA8/XXod8xrc+xWFxFjFC2hP4d0JcONMyhYa2RvmzbHXyN0rGZd6VY9Joooq0iiiigCkXDoaXTd3agGo0jtXUfSielNI0E0jSBXAPMPQ/lXENLA1nt+dAUftDwv8SRbua2gpYrJkty26CY7kdK8Z4hhLmIvviMVItW2yW7a6ZoPlRBqYjzE85r6A8OSSef2isHjuDjD3HuXHHhljE/wb5TPdQBHUd6ZPNeJcVt30VcKr58uXKQQCP6Y6TuY59ap/wC6TbElmJiHB+sD1Ea16nhuGWxZa7btKhZmZgQAc2u/RvTqKxXtPh3GINlNFYB8xO+adIjYEHmJrLd5aaeme4dYuNcAt23a4uoAieozE+UaDmdRy67TH8Vum3aF2yyXFJdQSoGZYy5WacyzrHaNqpsCt3CsS6uVaSXtOc8nfNABgz/Dt3qNxPGWbxLZoBB1zNcO4gEPrtM78qL3ROlp7VcbbEWkS9YVLiMCCGDE/wAy+TyxK9RED45fGcQtmybaIAxiWZRM6TBmRrVf41wEi27ZRzmP38KssHw7IC9xxJ0gSSrEn3p0BgTrtIqscZCyu0Lh4BzW3IAbn/KeRNbb2RweLvOFVl8NIDOy7Ab7EZmjWOQ370XARZUsz5gUYMUyhiVzBSDESdpMj3hqK9F9sfaNMNZ8HDWsiETKAKFVjOsbCTtzmnll+DGKDjnExhmPhuXgbNAB66S0aSdYNVVn20bzKlsa6SWgn08v7mqjCpcxbu2pfKQFVSQSdhoIXcdhVWbDKGV0hgYhh0kH1159qc/kVoMf7Q3FJm3LE6AuT3k5dqjWuL4q4fJaVfRXP/61+UVnzCsYPaY5flUj8UwXITpmDRoYI2iaKU89tK74gf8ANtozCQXAJ5bEFoGvSoTYy+6t/wCYeAygkELoQ5OixGy6/wBVM2ONm4Ar2s7rpmDFSR3AGumnKo3D8fbDszrKkTlVQRAjSGPYa9qjeWnbw/x7lNZdXXnrX6d4hZe2isbhYmW1LGVJGTf0Yx0jrVI7ljqSaveJ8QS5cDMMkDRAMxjkX2gxHlGwFQfBVklGzZQZEZTzMjUz1/2p4b/2c/z/AF8tfH4/n2r2AGh/e1dETtp67fKPrRk0BMx1qVYwynRSTOsdQAc0a69fhV2sdEoqu4WQJIE6n7zPpU24vgXGtpcD7QcpEyAYIMwe3arXBcBw5Us5M9PN9IEUrBXs5t2rdpXNsuWOUGdCA0bzlzGP5ipqN+1aXdmzcvYdbt1izsPLCqC2XKTtAOpIqovY23ctgK2s6Azy/mnlrHWtP7UcYFh7NiyqhLaKxBnSQsLMgiIn41jbPgy4UhizEggjTXWImRG3wqcNqyWPsvas28Qty5bL5DmyzJ0E5WGzbZg3PKBXqXHslyx49vVMskRyzA5gP50Ika7g15Jh2cXEMBshkmSJSRvEwNOvM6a1vvZjjqZPw96AICnYqJXeRuCNz84OlO3spNdvU7VwMoIIIIBBGxBGhHanaynsXxJfAey7rmw7MkkgDwwZRt/dC6T/AE1eYPiVq9m8N1fKcrQdjEifUa1pGaaDNKpK7UqgOUh+lYrxSCIJiPrT9nFuP42/7j+tZfZ+xp9f5WpC00dzXUuBlDDmBQ/barS6hovYpUZQSAWMAdf3FJzU08MYMA8ue20eh1pbGlgDUbFYUPIOoIgjSl4d8ygkQSNQeR5inqtKjuYXLmDhYOkkSrDaD3iBB6VhPajAorLdCQ6f8NgJHlk5W1nWT2mRXqrwQZiOc7R3rz/2s9osIFe2n/FYjw/LsCSIyvrJBg6AzoKnRyse5aBqSp1EfmOvL86p+I8FW5nKjKw3YbE/1gGd+fKrhS4BlcpGmvUbkfMbcjrXbWLa0C2UatDEiQDH+tFimVTg9y0pLpmykSRLAf4lGoJA06TUTjGIcZba5g0AtrzYe43JiojXetPxvFoV8ohtTuRBO40OxGx5a1lsdaKXG80ssOrzDQVDaxzAI1EazTlKxaYOzbs24YlSSnib5ioYFo55YIMfeqjHY26f+H5gigRqdVB0zH+JZEj0FN4w3B53csTzYydYnf0qJdxLfSPWP96McTtabgvGPDUAEW1KnzKJMzJnuTp0irPGWbbL4xt23YaMAQAWI7EKDrpGmmoBFMew9zAtba3iGKOW8tzUqCdljkfgAes09xTB4jKtqyM6F/I9uYC6kMGPuSN9Yk79Yy6yVO4yl/hxBnK0EwCDIBDRBP61HxCgvC+nz0qXYvORBcAM5WTvInpy1XedTyp+y1uUW6iyojykpmAB1JUEE9Toae7E62jYfCMltrkgELtI2JImCZ+XaoeFQgM4EsIjnA1lo5xA+dW15rTkG55R5B5DPl2YrmJlhppPen8XgktYi5h/4VmHB1KxKkr/AFSNtKJkLCPZzBW2UtcXOzbydvNB351LHCQpfwxsSwcb6QQCNucdfrUrC8KUY8WkZxaYkSAIMJmkZgQdhtzirLjF+5aP4e3ZQI0eY7MpYA+ZAIJJEnfXbnStOTTLvw4FjlaYXMxUEgGSGGg0iBvprTGCxDqQoyuk7jLI339an4jCM8taPmE+IFLHWSAu3QfUU3wXDGHuZFct5QGKwZ02mc0kQY0j1p73BrR2+7pbDWxkWBKtJOYnrERrO/61qP7PeGA4d7hWC5Os7mdG7do5eprF4nCoLfiKT5iAB0IOoPOZHKvUOFYpMJgrefXyqAFjUgDMfQDes8+po8e6Y9q8JYIYvDXbmiJvAUGWI9AZ9AN5rH4ngSIis4crnVfKBnG5YhYJIAG3P7TbvtGt4i4yhEjLbBddFUnQASzHUkxzmq/jXtHbdVS2GXUHNopBCnYnzHeNT1p4ywW7V74O9mcIwKOxBYhlB1IHcSCJnr2rWcP4Ei27ZuEKT5Zy+WczTmzHzEEQrZdQy9KzfDuLXh5M5KOCzAAEk/xROu/OrbG4pjbtXARkhJUqTkeRKuSdgQV21kHtVZW+CkL9qMOgvIqW1nMqh5kPA0LKsCYiPSdK9X9iODixhlliz3PO5O+oEL3yjST3rDYnCJcXxgQz+HA91VDHcxoCeUnbWtX7I8XY4S3JTSVgEGACRJIMGYnTkaUyKy1tKKp14q39J/frSv71/wAP7+NVzhcaz/gjrRkAqW+EYcpHbWoxQz3qjXnCLoa3l5rp8OX77VLunTSs5hH8O6rE6MrL8YzLP/aR/mqyvcVtzkzgNK6HvHTbQ84qbdeT1vwea9SrcN2YVxsu5Q9yDQttDqrEUghcP4xFx1c+VmlG5DQCD0Gkz3NTeN8ds4VM1xtToqjdjWeUZRBALEkR11NZv2n4N4rABvMsNzyrPSefl+1VN+isiu4/7ZXMYXt5hbtDdBOo00aIZ2MkhR05EVmLateu5LCkMAV8QsAzEtIiRltkkCOfcTXMU6W0dCGzqYBjQRMRGyEadZiqjh966jlrRgsI90ER8RS7HT1b2TvuRdwmNDwi2/DDjIQhLgHyQcxI3123NSOJ+yvlZsNcDEyfDdogn+QnSY6/Oq32f9tLeX/zLlHGh8QBgY00YCVH9P1OprUYbimEurKlW72yNfkfL8TS5UcXl+Pw1yySl1GRtwGH/uHJonlNUHEiWlzPiyxM7mdxHxNe7nhFq8AfEuZfejxDoR8eXUV5xxrA27t25o6sHYW3c6yuyuW8zA9zz7U5Qwt28zoRlgLp5uvIbabzUPNIj+LmBz+H79K0PGOBYhLRcqHtqFzOhLAakANIBmWjSREVnImOTcjO/Y9Oxq5U0/YvZPNl1ggMDBU7En4E/GrjC8fvLbyTmXl5iCN+x68oqFh+IWyAt22uYEDPG6zqHjUH+ofKnxw5cgPiDXZ18y+hCmQZG9RlZ7Xjv0T+NWVL2UfLzEKe8iMrn1GtC2luZFzZEkKWaM0E6zBj9ac4dw3xC4BRiqg+8dyyidtonrrRxXh1yxlFyBmEwpnmR06ilMpvQuN8qy5bOZj/AMvMUVuWhkROo0E1MxGKNxy7jzsiqQjAxlyqNIMsQu08x6U4/DWKq6uMjcj8QYXXv3qVhOBXrgbwlyncsSoZhzyawusCJG+pp2wtVc4G4zOptoItzkZtwdsgbkCDBjp3MLD3TdJvBgHuJkRBIMHyqIbTQ9+Zqbw7BXrdtEbCXUySM5YchyAfv6b13G4xbTqxuZChYxpockRqpGxI1NY3PvTWY9IFtLFtCmaTcGeMoYjUgAE/DY/6QuE8JuNcNtXZFDAlgWjUSABPmYaa6az0qdwbCvcYs1oMttDlcAKBCzAIgMYG+pqThcY4tgZxZYiX28TNsVAI8gERoCdJ50XKzwXEvDYe3Ya4jWyjg+U50e65OhYgAraBMd4NV/tTiri28rZVdlKrbUklUb3i2uhMkzzntXfx7rm8NESZ87nM5nY8gG35azzqDbR2bMTbdgZIddSTtJBBn40p3d09ddM3awZgHSTy3O8ajlVnwzHGyzgpnLAAgMF1Uk7kHkTI7VpXxVwSBZw6/wCBFkaawW595+NZnE2JZnZlBkmAI31Ecjz5mtZltHHRu7xYyMqAPqSSSYLFvIQfeBEfPtU/AYzOrsfKDusSA2slZ2/Kap2LsA3Lv/tUq28KdiTtuNdeo13HOndJm19wrFFlCXACi3GuEZdSRbcZjHLSIM9a33stbs/hUDW0JOY+ZVJ949fSvMMXilFxxbaBmjLMEaQRPPWRXo3BEH4e1/gU7dRP3NYZZdS/rXDHd1Wi/D4eNbVsDvbT9KV4OH/6Vv8A7E/Sq1UB05CnPw46xWf2Vp9cTkxBXYxSnxpPvajtH5imhboTDyfzrrtc8jt0WXU5iyRrmDAQeuoIBqr4v7M4bEul23fa3ctwQ6lHJKkFWfNJYggevOrG9h1OhE1U3+C2Cc3hiQZB2IPYiotu16i/GKvZdPBcgwSHK8t8rbE9M2k76VBb2geyx8eyyKNc6yVj1IA+TGs/ew93VrdxgDuHBE/Y9telM4fHXEzBgxA1JR4gc5hgT6Ut9+U9zzFnY4qly/4hdQgLCYIDec5cs7GGk+hpN7iKZ3zaBtRr3IA/fWsxZxrNduI0nVWDAe7MZh1JIX70jjauba5R5tVJ6dSPiCK1xy0Vm3ONcNV2LkanWeVRkQWiRb99AdxIYxroKlcPW4LJt3CSJBBbU6ZSQCfj6UxewhtgEsPFYZwoE+U7E8pOta3OWImNlVWIQHXqJiOv5VUYxApzI+VhzUwfTTUVcWrue4/iJOm2vY+WPe0P0qLjeD28rPbcDMPdJEdQR8Y+dZ2zatXRvh3tdi7JEXCwHJ9frv8AOa0HDvbIybkBmZszJkDEaEkpMZufMHXasU+EJgqQ3IxyPx3HpTz8KbKCGEnroPgTzHpStxOcnqeK41YuKqIQoZQzgDJuB740g9RVRjOBW7jhHCMjbON0iBGZeUQdaxirirYjPmUDrMbfziQPpVlgeI4sqBOUTM6ZviQAIpat7gup5V3tLwr8Pfayzq2WMrDmCNAwnysPXpyIirs27inODkjmdD303PpV/iWto7XLhz3GJZv4jJ9dqrnv5iWChZ2JMk9u2+9X48p8rawDbtOXvSfL5fKCR5pIMSInvy6Qam5xa2VZfCLEsCpZyY6zABM/CoV1CTLb0g2qjU2rtbJiFNuTcVWK6AR5dxEDbXXrqKtPZvHXreUsCUbRGJJABBDDU7aDmBI71k/Cq04Xxa7YGQZXtkybbgld5lYIZD3UilljuDG6rcY/E4tQwB8mZgCSZOp1AOg2iZjWq1cdi1AHieIsaKwDgHNMQwIBnnrUjhvt3bRBntOXX3SWRwND2Uk9z9aq+M+1q3INu3ctwzMIdVidwIQkCdYkTWUws9NOUS39q8YphssCWKlVg7zMDeB6DoKxpxBUAK7Ceh0+pGtS8Xxu9dVkJgMSXI1Zp5Mx5abCJ5zUBLUcpPLoP1Na4468oyy34PI5gsZJkiCeUTIn7VFW4B85qyx+BdEtTuyF50PvMcu23lAqMuDZuQ76faq3E6qZhuJhQJtiB3P6V29xjNm8gAJB3nUR21GkR0qEcAT2HQU6mEI5mO9TrFXbt3iJYZSBAkgRtMzGvem7V558pae359am4ewBugNWnh2nTKbYWNmURB7x73oaqSJtquwwze9bWQNQoIMczHOtd7NcT8Mi2M5RgSoYEMCNxMwV+RqJwfCJbfPbuBGO8ohB7ERtWlQXLmXO1vKCfdRlJBPKXb99Kzyx31peN0srPEk5q3wA/WpH952+h+QqKmCQjSfnSv7uH85+QrO/EvnF97g18wG/UfLeutcnURHanHeTpUa7bkyNDr8fWt2cIfWmWB66U5mMa6HnzonoKmqQ3t/H1qHesjfKCatGYzypp6zs2qVk8cnhXbd2IUMFbt5gVPpMr/nqy41dS2wYCUK+JIAOknlsSYY9s1Ssfh0uIUbUNIP6+tZy1mtzauqTlBVW5On2zDpyqseoWXlWcU4619swIXL7qLyXt121qw4oTct27tqXyjXqZiZHXQzWc4pw0WjmtuSP4TBBI5g9GHP0qx9nsYottLkNO3wGo033q8r7icfyqLiONzXS2XIVMALoB3PMk96bs3ZaXkz3n6VZ+0Fm0W8RHGZjqp0Hcg7RPLuar8M9sTLrMde3b9+tOSWbK7l0scNbtgl4gkRtpG8+tP3uIWoVWKoukk+9B6DkY61WYji1q2hFtQznbmF79DUPhOGuXX0gsTqzHrOp+tEwk7pXK+IssTirYaEBuMdRBBXfmwPT78q5ibGINs3CcqLqQs6A8ydyKu+E+yV11uOWCeGTGxzMACMuo8p2zbfai1irgsuC6FMpGpgjlERqdYouX4JGVw+ELDMX0Os5dT8967dJJGVWI7W/1q+weAuP7rpl/pE/nAqys8LZfebWCeUR6mpuRzGspa4ez+aWXlqPvlpvFXEDZQmYDqCD9f0rVJhnbRLXl11A09QBrXb3B2ckvbUN1nf4E0uR8WSGKX/piOx/0pu7lf3VOb51qLHAhIm27DpoPyqzucOFoTaKq38rBfuBvRykGqxowLrbzspPPQQfjUNLbudLZI6D79q174fGnzD0MZYpp8LibnkKkjfQD7iiUWKe3wo5ZIyjuR+RqVb4RmGhU8pEEferzh/BGtnNd8NRGoZpMduVW6YVGRvDIIOvMCZ5ZR9Km7VJGXt8JaROsADaNtPyqQvCTp5Y/f3q7w2ExAOq2gPRyd/WrRLM8ojc8p6Cldn0yA4bb5soPSRTw4MK0GM4dcb3LjjbRQo+rQab/wDDjHXx7gPPX8poko3FIOEDpXRwQnaRFX2ES1alXvhyDHmBBH61bWbSNBHmHWN/SjVLcZbD8HYetXGFwZG5q5WwBypxbXYVeONTajW7cc6czV24WGyg+pAprxn/AOmPn/pVpXIImY+dcuP3EU02u9Gm5+tAczdKjsmU9o23jvUg9qFt+k/b1paPZgQdjpTdzQHWB12+M0+uG7xr8B+tMX8MMwLAHoeU8tCd6Wj2ieWJG28gfuai4hZEECOYMfnVrcw8Hff9ioz4QyfMfTSB9N6nSts1i8IjAyg15R9TVJieF2pPkg9QWH2Nbi7hTOm3cVEv8NBGtLdh6lYDEcMtGQwYb+YHb4TrVNicILZGUhpWCRMBpPXqI+tek3eDCe3p+9KZu8AUiCN9NqczqbjHnlmwACxGp0E7d6tuDKyglWysIIaAY0adDvvVvjvZdkIAR2UkGEjy6jcHr0HfWmzw5gScjg/ylY07akGtJYixoeD4drutzFO8gaLlX6RPKr1vZuy+rAlojzEH5iN+9U/COGlVA7z21M1o8NcZNjIHLf4dqOh2ipwDwxCAR02P1qBf4IJJYuNZ00+XUVpkxk+9p6fpUkZX2Yd/9ZpcYfKxk34f5QLblD11Y/U6UzhbKKSruHbrAEdh3rYtw+2RqI9NKgYnCLbBZRmjoNam4nMtqK7wxm2c+mvyqOlgW/8A1CY/qAI+EnWu4njFwEhbRidzLfbtSWuDEJ50ZWWNUDN9I/OjStkrjMNrJcdpgfCDTuCxluDlZj2MMfXQbfGo6+z4YTm0/qEH5TpU7CezyI0lwRttt6if9qeonZacOtli7qrM2skVLNhYiNOgANFjA2lMhQNYE8vSpbWzHk0PdSfzFGhszbsjbbtXXtNHkyz/AFDT6VMtKYhoJ9I+Qk1y9YJGjQaNFtHs22jzZZ7CB8qcW0OQEn0pkYV13umPQVIcGNDPrz/KnIRt8Op3An0APzpSWADI39SfuajJfuTqkfCptt55RT1Bt0T1rjMZ2FOMO5Hp+f6VwJ3+cUaLZtkB/wBqb/DjvUiR1rmUdfvRoHOvp+dKfnRRQZFvdac5L/m+9FFAIP7+tcO1FFARLHvP6inD+f5UUUoo3d94VFxHL0P5VyilThqx+X61JTc/vkaKKUKnbfP/AA01d5ev5UUVXoJTKJOnM100UUwSnKnDRRQEgsdNeZpb7/L70UUGXgKpeI/+ofU0UUFPKuHvj4/arSx7qeh+9FFBVItjWltRRQRfL50LyoooB07fCq5/ertFKhKXnSMPv8T+dFFVE05yb0ppeXofsK5RQCjv8T96VRRVFX//2Q==',ft_model)

1/1 [==============================] - 1s 641ms/step
Validating severity of damage....Result: severe
Severity assessment complete.


In [32]:
pipe4('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTAWFgj6xctNM_s2R1YVc8KFgwwpt_bjFvOzAbkJ3L4RpKjJg-rYRPFdlhfO8G42bL61Mc&usqp=CAU',ft_model)

1/1 [==============================] - 1s 605ms/step
Validating severity of damage....Result: minor
Severity assessment complete.
